# In the transaction_candidate_list, we want to find transaction with known change revealed by multi-agent heuristics

In [1]:
import json
import os
from tqdm import tqdm
import os
import requests
import pickle
import matplotlib.pyplot as plt
import numpy as np

## load data

In [2]:
# path of blocks
SOURCE_PATH = '../data/txn_data'
# SOURCE_PATH = 'testData'
# read files 
block_list = os.listdir(SOURCE_PATH)
sorted_block_list = [y for _, y in sorted([(int(a.split('.')[0]), a) for a in block_list])]
sorted_block_list = sorted_block_list
# print(sorted_block_list[:10])
print(f"There are {len(sorted_block_list)} blocks loaded")


def get_block_txn_list(block_json_file: str) -> dict:
    """
    @input: a block json file, e.g., 1111.json
    @output: a list of transaction in that block
    """
    json_path = os.path.join(SOURCE_PATH, block_json_file)
    f = open(json_path)
    json_data = json.load(f)
    tx = json_data['tx']
    f.close()
    return tx

There are 4087 blocks loaded


In [3]:
ADDRESS_REUSE_CODE = 0
CLUSTER_MEMBER_CODE = 1
UNKNOWN_CHANGE_CODE = 2
OVERLAY_APPLICATION_CODE = 3
TWO_OUTPUT_CODE = 2

with open('../heuristic_data/address_reuse_transaction_address_list.pkl', 'rb') as f:
    address_reuse_transaction_address_list = pickle.load(f)

################################################################################################

with open('../heuristic_data/overlay_application_transaction_address_list.pkl', 'rb') as f:
    overlay_application_transaction_address_list = pickle.load(f)

################################################################################################

with open('../heuristic_data/transaction_candidate_list.pkl', 'rb') as f:
    transaction_candidate_list = pickle.load(f)

################################################################################################
with open('../heuristic_data/transaction_address_summary.json') as f:
    transaction_address_summary = json.load(f)

################################################################################################
with open('../heuristic_data/two_output_address_summary.json') as f:
    two_output_address_summary = json.load(f)

print(f"number of transactions with reused addresses issue: {len(address_reuse_transaction_address_list):,}")
print(f"number of transactions with overlay application issue: {len(overlay_application_transaction_address_list):,}")
print(f"number of candidate transactions: {len(transaction_candidate_list):,}")

number of transactions with reused addresses issue: 1,231,860
number of transactions with overlay application issue: 16,165
number of candidate transactions: 4,143,799


## implement multi-agent heuristic clustering method

### create an address book

In [4]:
# def get_all_address_():
#     """
#     @return: 
#     return the sender address and receiver address of all candidate transactions
#     """
#     local_address_book = []
#     for block_json_file in tqdm(sorted_block_list):
#         # get transaction list of that block
#         txns_list = get_block_txn_list(block_json_file)
        
#         for txn in txns_list:
#             txn_hash = txn["hash"]
#             if transaction_address_summary[txn_hash] == TWO_OUTPUT_CODE \
#                     and two_output_address_summary[txn_hash] != ADDRESS_REUSE_CODE \
#                     and two_output_address_summary[txn_hash] != OVERLAY_APPLICATION_CODE:
                
#                 for input_ in txn["inputs"]:
#                     local_address_book.append(input_["prev_out"]["addr"])
                
#                 for output in txn["out"]:
#                     if "addr" in output:
#                         local_address_book.append(output['addr'])
#     return local_address_book


# address_book = list(set(get_all_address_()))
# """address_book is a list of the address of all senders and receivers"""
# print(f"There are {len(address_book):,} candidate address")

In [5]:
# with open('../heuristic_data/address_book.pkl', 'wb') as f:
#     pickle.dump(address_book, f)

with open('../heuristic_data/address_book.pkl', 'rb') as f:
    address_book = pickle.load(f)

print(f"There are {len(address_book):,} candidate address")

There are 11,341,115 candidate address


### initialize the clustering dictionary of addresses in transactions in transaction_candidate_list

In [6]:
multi_agent_input_clustering = {}
"""
key: transaction hash of each transaction in transaction_candidate_list
value: clustering id; if there's no clustering assigned, clustering id is -1
"""
for address in tqdm(address_book):
    multi_agent_input_clustering[address] = -1

100%|██████████| 11341115/11341115 [00:03<00:00, 2857364.67it/s]


### Count how many times an input / output address appeared in candidate transaction
- Store results in count_clustering_candidate hash map

In [7]:
# count_clustering_candidate = {}

# # initialize the clustering hash table
# for address in tqdm(address_book):
#     count_clustering_candidate[address] = 0

# for block_json_file in tqdm(sorted_block_list):
#     # get transaction list of that block
#     txns_list = get_block_txn_list(block_json_file)

#     for txn in txns_list:
#         txn_hash = txn["hash"]
#         if transaction_address_summary[txn_hash] == TWO_OUTPUT_CODE \
#                 and two_output_address_summary[txn_hash] != ADDRESS_REUSE_CODE \
#                 and two_output_address_summary[txn_hash] != OVERLAY_APPLICATION_CODE:
#             input_list = []
#             output_list = []
#             for input_ in txn["inputs"]:
#                 input_addr = input_["prev_out"]["addr"]
#                 count_clustering_candidate[input_addr] += 1

#             for output in txn["out"]:
#                 if "addr" in output:
#                     output_addr = output['addr']
#                     count_clustering_candidate[output_addr] += 1


# with open('../heuristic_data/count_clustering_candidate.json', 'w') as f:
#     json.dump(count_clustering_candidate, f)

with open('../heuristic_data/count_clustering_candidate.json') as f:
    count_clustering_candidate = json.load(f)

### Step 1: Merging all inputs together

In [8]:
clustering_candidate = {}
change_address_reuse_list = []

# initialize the clustering hash table
for address in tqdm(address_book):
    clustering_candidate[address] = -1

100%|██████████| 11341115/11341115 [00:03<00:00, 3516849.55it/s]


In [9]:
count_lonely = 0
current_clustering_index = 0

for block_json_file in tqdm(sorted_block_list):
    # get transaction list of that block
    txns_list = get_block_txn_list(block_json_file)
    
    for txn in txns_list:
        txn_hash = txn["hash"]
        if transaction_address_summary[txn_hash] == TWO_OUTPUT_CODE \
                and two_output_address_summary[txn_hash] != ADDRESS_REUSE_CODE \
                and two_output_address_summary[txn_hash] != OVERLAY_APPLICATION_CODE:
            
            input_list = []
            output_list = []

            ######################################################################
            # store inputs and outputs
            for input_ in txn["inputs"]:
                input_list.append(input_["prev_out"]["addr"])
            for output in txn["out"]:
                if "addr" in output:
                    output_list.append(output["addr"])
            ######################################################################

            ######################################################################
            # check if none of its input/output appear anywhere else
            # lonely bit = 1 if it appears anywhere else
            lonely_bit = 0
            for address in set(input_list).union(set(output_list)):
                if count_clustering_candidate[address] != 1:
                    lonely_bit = 1
                    break
            
            if not lonely_bit:
                count_lonely += 1
                continue
            ######################################################################

            ######################################################################
            explored_bit = 0  # 1 if any of the input is explored before 
            explored_cluster_id_ = None
            for input_ in input_list:
                if clustering_candidate[input_] != -1:
                    explored_bit = 1
                    explored_cluster_id_ = clustering_candidate[input_]
                    break
            ######################################################################

            if explored_bit:
                for input_ in input_list:
                    clustering_candidate[input_] = explored_cluster_id_
                # check if it's discovered before
                for output in output_list:
                    if clustering_candidate[output] == explored_cluster_id_:
                        change_address_reuse_list.append(txn_hash)
                        break
            else:
                for input_ in input_list:
                    clustering_candidate[input_] = current_clustering_index
                current_clustering_index += 1

100%|██████████| 4087/4087 [07:00<00:00,  9.71it/s]


In [10]:
with open('../heuristic_data/clustering_candidate.json', 'w') as f:
    json.dump(clustering_candidate, f)

# with open('../heuristic_data/clustering_candidate.json') as f:
#     clustering_candidate = json.load(f)

### Step 2: Filter transactions where the change has been revealed by the multi-input heuristics

In [11]:
clustering_member_list = []
unknown_change_list = []

In [12]:
for block_json_file in tqdm(sorted_block_list):
    # get transaction list of that block
    txns_list = get_block_txn_list(block_json_file)
    
    for txn in txns_list:
        txn_hash = txn["hash"]
        if transaction_address_summary[txn_hash] == TWO_OUTPUT_CODE \
                and two_output_address_summary[txn_hash] != ADDRESS_REUSE_CODE \
                and two_output_address_summary[txn_hash] != OVERLAY_APPLICATION_CODE:
            
            input_list = []
            output_list = []
            
            ######################################################################
            # store inputs and outputs
            for input_ in txn["inputs"]:
                input_list.append(input_["prev_out"]["addr"])
            for output in txn["out"]:
                if "addr" in output:
                    output_list.append(output["addr"])
            ######################################################################

            input_cluster_id = clustering_candidate[input_["prev_out"]["addr"]]
            clustering_member_bit = 0
            for output_addr in output_list:
                if clustering_candidate[output_addr] != -1 and clustering_candidate[output_addr] == input_cluster_id:
                    clustering_member_bit = 1
                    break
            
            if clustering_member_bit:
                clustering_member_list.append(txn_hash)
            else:
                unknown_change_list.append(txn_hash)

100%|██████████| 4087/4087 [07:55<00:00,  8.59it/s]


In [13]:
print(f"There are {len(clustering_member_list):,} cluster member transactions")
print(f"There are {len(unknown_change_list):,} unknown change transaction")

There are 140,828 cluster member transactions
There are 4,002,971 unknown change transaction


In [14]:
with open('../heuristic_data/clustering_member_list.pkl', 'wb') as f:
    pickle.dump(clustering_member_list, f)

with open('../heuristic_data/unknown_change_list.pkl', 'wb') as f:
    pickle.dump(unknown_change_list, f)

with open('../heuristic_data/clustering_candidate.json', 'w') as f:
    json.dump(clustering_candidate, f)

In [15]:
len(clustering_member_list)

140828

In [16]:
with open('../heuristic_data/change_address_reuse_list.pkl', 'wb') as f:
    pickle.dump(change_address_reuse_list, f)